A Bloom filter is a data structure that is used to check for membership of an element x in a set of m elements.

http://prakhar.me/articles/bloom-filters-for-dummies/

http://axiak.github.io/pybloomfiltermmap/

Download from:
https://pypi.python.org/pypi/pybloom/1.0.2
pip install ez_setup
python setup.py install

In [1]:
!pip install pybloom

  Using cached https://files.pythonhosted.org/packages/1a/82/a1ad015bdc19bd7e10aa97070329b84b5e01c0c6b5de88df664a98413eed/pybloom-1.1.tar.gz
  Running setup.py bdist_wheel for pybloom: started
  Running setup.py bdist_wheel for pybloom: finished with status 'done'
  Stored in directory: C:\Users\Sam\AppData\Local\pip\Cache\wheels\19\1d\a7\51ab423bf9609665cfc365559c62c652d5b4e414a6faad5ed6
Successfully built pybloom


distributed 1.21.8 requires msgpack, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from pybloom import BloomFilter
f = BloomFilter(capacity=10000, error_rate=0.001)
for i in xrange(0, f.capacity):
    _ = f.add(i)
0 in f

True

In [3]:
f.capacity in f

False

In [4]:
len(f) <= f.capacity

True

In [5]:
 abs((len(f) / float(f.capacity)) - 1.0) <= f.error_rate

True

In [6]:
from pybloom import ScalableBloomFilter
sbf = ScalableBloomFilter(mode=ScalableBloomFilter.SMALL_SET_GROWTH)
count = 10000
for i in xrange(0, count):
     _ = sbf.add(i)
sbf.capacity > count



True

In [7]:

len(sbf) <= count

True

In [8]:
abs((len(sbf) / float(count)) - 1.0) <= sbf.error_rate

True

#Using a bloom filter on a set of posts

http://www.stavros.io/posts/bloom-filter-search-engine/


In [9]:
from pybloom import BloomFilter
import os
import re

POST_DIR = 'data/posts/'
# Read all my posts.
posts = {post_name: open(POST_DIR + post_name).read() for post_name in os.listdir(POST_DIR)}
# Create a dictionary of {"post name": "lowercase word set"}.
split_posts = {name: set(re.split("\W+", contents.lower())) for name, contents in posts.items()}

In [10]:
filters = {}
for name, words in split_posts.items():
    filters[name] = BloomFilter(capacity=len(words), error_rate=0.1)
    for word in words:
        filters[name].add(word)

In [11]:
def search(search_string):
    search_terms = re.split("\W+", search_string)
    return [name for name, filter in filters.items() if all(term in filter for term in search_terms)]

In [12]:
search("nothing")

['1.txt', '2.txt']

In [13]:
search("ice")

['3.txt', '4.txt']

In [17]:
# %load data/posts/3.txt
Vacation Time


SyntaxError: invalid syntax (<ipython-input-17-7b2b60e7aa12>, line 2)

In [15]:
# %load data\posts\4.txt
Ice cream is always good

SyntaxError: invalid syntax (<ipython-input-15-6919083190f7>, line 2)

In [18]:
 sum(len(filter.bitarray.tobytes()) for filter in filters.values()) / len(filters)

6